###  Develoing a small LSTM network

In [108]:
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [81]:
import numpy as np

In [95]:
from keras.layers import Input

In [97]:
from keras.models import Model

In [98]:
from keras.optimizers import Adam

In [4]:
#UTF-8 to read the ASCII text
with open('AliceInTheWonderLand.txt','r',encoding='utf-8') as f:
    RawText = f.read()
    

In [29]:
rawtext = RawText.lower() #coverting all characters into lower characters

In [30]:
unniqueCharacters = set(rawtext)

In [31]:
unniqueCharacters #set of all unique characters

{'\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [32]:
chars = sorted(list(unniqueCharacters))

In [33]:
chars

['\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [34]:
charToInt = dict((char,index) for index,char in enumerate(chars))
#giving index value to characters

In [35]:
charToInt

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '#': 4,
 '$': 5,
 '%': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 '*': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '?': 27,
 '@': 28,
 '[': 29,
 ']': 30,
 '_': 31,
 'a': 32,
 'b': 33,
 'c': 34,
 'd': 35,
 'e': 36,
 'f': 37,
 'g': 38,
 'h': 39,
 'i': 40,
 'j': 41,
 'k': 42,
 'l': 43,
 'm': 44,
 'n': 45,
 'o': 46,
 'p': 47,
 'q': 48,
 'r': 49,
 's': 50,
 't': 51,
 'u': 52,
 'v': 53,
 'w': 54,
 'x': 55,
 'y': 56,
 'z': 57}

In [41]:
n_chars = len(rawtext)
n_vocab = len(chars)

In [42]:
print("Total Raw text length ===> {}".format(n_chars))
print("Total Characters length ===> {}".format(n_vocab))

Total Raw text length ===> 163779
Total Characters length ===> 58


In [76]:
seq_length = 100
dataX = []
dataY = []
count = 0
for i in range(0,n_chars - seq_length,1):
    seq_in = rawtext[i:i+seq_length]
    seq_out = rawtext[i+seq_length]
    dataX.append([charToInt[char] for char in seq_in])
    dataY.append([charToInt[seq_out]]) #adding the 101th letter of the character
n_patterns = len(dataX)
print("Total PAtters ==> {}".format(n_patterns))

Total PAtters ==> 163679


In [77]:
seq_in,seq_out

('o help produce our new ebooks, and how to\nsubscribe to our email newsletter to hear about new ebooks',
 '.')

In [80]:
dataX[0],dataY[0]

([47,
  49,
  46,
  41,
  36,
  34,
  51,
  1,
  38,
  52,
  51,
  36,
  45,
  33,
  36,
  49,
  38,
  7,
  50,
  1,
  32,
  43,
  40,
  34,
  36,
  7,
  50,
  1,
  32,
  35,
  53,
  36,
  45,
  51,
  52,
  49,
  36,
  50,
  1,
  40,
  45,
  1,
  54,
  46,
  45,
  35,
  36,
  49,
  43,
  32,
  45,
  35,
  11,
  1,
  33,
  56,
  1,
  43,
  36,
  54,
  40,
  50,
  1,
  34,
  32,
  49,
  49,
  46,
  43,
  43,
  0,
  0,
  51,
  39,
  40,
  50,
  1,
  36,
  33,
  46,
  46,
  42,
  1,
  40,
  50,
  1,
  37,
  46,
  49,
  1,
  51,
  39,
  36,
  1,
  52,
  50,
  36,
  1,
  46,
  37],
 [1])

##### 1 in dataY is the next character to come after 37, which is the last index (-1) value on dataX

In [87]:
# reshaping X to (samples,time_steps,features)
X = np.reshape(dataX,(n_patterns,seq_length,1))

In [88]:
X.shape

(163679, 100, 1)

In [90]:
X = X/float(n_vocab)

In [91]:
X

array([[[0.81034483],
        [0.84482759],
        [0.79310345],
        ...,
        [0.01724138],
        [0.79310345],
        [0.63793103]],

       [[0.84482759],
        [0.79310345],
        [0.70689655],
        ...,
        [0.79310345],
        [0.63793103],
        [0.01724138]],

       [[0.79310345],
        [0.70689655],
        [0.62068966],
        ...,
        [0.63793103],
        [0.01724138],
        [0.55172414]],

       ...,

       [[0.01724138],
        [0.87931034],
        [0.79310345],
        ...,
        [0.56896552],
        [0.79310345],
        [0.79310345]],

       [[0.87931034],
        [0.79310345],
        [0.01724138],
        ...,
        [0.79310345],
        [0.79310345],
        [0.72413793]],

       [[0.79310345],
        [0.01724138],
        [0.67241379],
        ...,
        [0.79310345],
        [0.72413793],
        [0.86206897]]])

In [92]:
y = np_utils.to_categorical(dataY) #one hot encoding

In [93]:
y

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [94]:
y.shape

(163679, 58)

In [96]:
X.shape

(163679, 100, 1)

In [103]:
def Baseline():
    inpt = Input(shape=(X.shape[1],X.shape[2]))
    x = LSTM(256)(inpt)
    x = Dropout(0.2)(x)
    classify = Dense(y.shape[1],activation='sigmoid')(x) #to return probability of all the classes
    model = Model(inputs=inpt,outputs=classify)
    adam = Adam(learning_rate=0.002,decay=0.0001)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [104]:
model = Baseline()

In [105]:
filepath = 'weights-improvement-{epoch:02d}-{loss .4f}.hdf5'

In [110]:
checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=1,save_best_only=True,mode='min')

In [111]:
callback_list = [checkpoint]

In [114]:
model.fit(X,y,epochs=20,batch_size=128,callbacks = callback_list)

Epoch 1/20
 36992/163679 [=====>........................] - ETA: 9:39 - loss: 3.1214 - accuracy: 0.1682

KeyboardInterrupt: 